In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.pool import global_mean_pool

In [2]:
from helpers import CVFConfigDataset

from torch.utils.data import DataLoader, random_split

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
torch.manual_seed(20)
# for same weight re-initialization

In [5]:
class GCNConvByHand(nn.Module):
    """maps D x N to D x N"""

    def __init__(self, dim_in):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_in, bias=True)
        # torch.nn.init.ones_(self.linear.weight)
        # torch.nn.init.ones_(self.linear.bias)
        # torch.nn.init.xavier_uniform_(self.linear.weight) 

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k.shape, "beta_k", beta_k)
        # H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes).to(device), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(x, A + torch.eye(num_nodes).to(device)))
        return x

In [6]:
class GCNByHand(nn.Module):
    def __init__(self, N, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels)
        self.conv2 = GCNConvByHand(in_channels)
        self.linear1 = torch.nn.Linear(N, 64, bias=True)
        self.out = torch.nn.Linear(64, out_channels, bias=True)

    def forward(self, x, A):
        x = self.conv1(x, A)
        x = torch.relu(x)
        x = self.conv2(x, A)
        x = torch.relu(x)
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.out(x)
        x = torch.relu(x)
        # x = torch.sigmoid(x) * 9
        # print("x after output layer\n", x)
        # x = global_mean_pool(x, torch.zeros(x.size(1)).long())  # all are from graph 0, single graph
        return x
    
    def fit(self, train_loader, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.01) # weight_decay is a L2 regularization parameter
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in train_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                optimizer.zero_grad()
                out = self(x, train_loader.dataset.dataset.A.to(device))
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Training set | Epoch:", epoch, "Loss:", total_loss / count)
            
            # self.eval()
            # with torch.no_grad():
            #     total_loss = 0
            #     count = 0
            #     for batch in validation_loader:
            #         x = batch[0].to(device)
            #         x = add_graph_properties(x)
            #         y = batch[1].to(device)
            #         y = y.unsqueeze(0).reshape(-1, 1, 1).float()
            #         optimizer.zero_grad()
            #         out = self(x)
            #         loss = criterion(out, y)
            #         total_loss += loss
            #         count += 1

            # print("Validatn set | Epoch:", epoch, "Loss:", total_loss / count)
            # print()

In [7]:
num_nodes = 8       # N
num_features = 1     # D
num_labels = 1       # O
batch_size = 10      # B

# the following needs to be replaced by some node embeddings
# x = torch.randn((batch_size, num_features, num_nodes)) # B x D x N

# A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(N=num_nodes, in_channels=num_features, out_channels=num_labels)
model.to(device)

# out = model(x, A)   # B x D x O

# print("output", out.shape)

GCNByHand(
  (conv1): GCNConvByHand(
    (linear): Linear(in_features=1, out_features=1, bias=True)
  )
  (conv2): GCNConvByHand(
    (linear): Linear(in_features=1, out_features=1, bias=True)
  )
  (linear1): Linear(in_features=8, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)

A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
# preactivation

out_wt = np.array(model.out.weight.detach().numpy())
out_bias = np.array(model.out.bias.detach().numpy())
preactivation @ out_wt.transpose() + out_bias

In [8]:
# dataset = CVFConfigDataset(
#     "dijkstra",
#     "implicit_graph_n10_config_rank_dataset.csv",
#     "implicit_graph_n10_A.json",
#     3,
#     one_hot_encode=False,
# )
dataset = CVFConfigDataset(
    "coloring",
    "graph_random_regular_graph_n8_d4_config_rank_dataset.csv",
    "graph_random_regular_graph_n8_d4_A.json",
    5,
    one_hot_encode=False,
)

In [9]:
len(dataset)

390625

In [10]:
train_split = 0.95
test_split = 0.05

train_set, test_set = random_split(
    dataset, [train_split, test_split]
)

train_loader = DataLoader(train_set, batch_size=1000, shuffle=True)
test_loader = DataLoader(test_set, batch_size=10, shuffle=False)

In [11]:
train_loader.dataset.dataset.A

tensor([[0, 1, 0, 1, 0, 1, 0, 1],
        [1, 0, 0, 1, 1, 0, 1, 0],
        [0, 0, 0, 1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 0],
        [1, 0, 1, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 1, 0, 1],
        [1, 0, 1, 0, 0, 1, 1, 0]])

In [12]:
model.fit(train_loader, 20)

Training set | Epoch: 1 Loss: tensor(1.3683, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 2 Loss: tensor(1.2092, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 3 Loss: tensor(1.2098, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 4 Loss: tensor(1.2031, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 5 Loss: tensor(1.1991, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 6 Loss: tensor(1.1959, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 7 Loss: tensor(1.1936, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 8 Loss: tensor(1.1941, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 9 Loss: tensor(1.1913, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 10 Loss: tensor(1.1915, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 11 Loss: tensor(1.1934, device='cuda:0', grad_fn=<DivBackward0>)
Training set | Epoch: 12 Loss: tensor(1.1919, device

In [13]:
# testing
torch.no_grad()
# torch.set_printoptions(profile="full")

total_matched = 0

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    predicted = model(x, test_loader.dataset.dataset.A.to(device))
    print(y[0], predicted[0])
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched

print("Total matched", total_matched, "out of", len(test_set), "| Accuracy", round(total_matched/len(test_set) * 100, 4), "%")

tensor([[1.]], device='cuda:0') tensor([[3.3391]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[3.]], device='cuda:0') tensor([[3.0678]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[4.]], device='cuda:0') tensor([[3.0952]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[2.]], device='cuda:0') tensor([[2.8008]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[2.]], device='cuda:0') tensor([[3.4533]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[4.]], device='cuda:0') tensor([[3.0948]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[3.]], device='cuda:0') tensor([[2.8766]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[2.]], device='cuda:0') tensor([[2.9614]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[3.]], device='cuda:0') tensor([[2.7433]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[4.]], device='cuda:0') tensor([[3.7729]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[3.]], device='cuda:0') tensor([[2.5296]],